# Imports

In [1]:
from nba_api.stats.endpoints import leaguegamelog
import model.dataset.data as data
import model.dataset.config as config

import model.dataset.season_game as sg
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)

In [ ]:
from nba_api.stats.endpoints import leaguedashteamstats

In [ ]:
leaguedashteamstats.LeagueDashTeamStats().get_data_frames()[0]

In [ ]:
rankings = data.load_rankings()

In [ ]:
list(raw_season_games.columns)

# Load local dataset

## Raw Season Games

In [2]:
raw_season_games = pd.read_feather(config.RAW_SEASON_GAMES_DS)
raw_season_games.set_index(["GAME_ID"], inplace=True)
raw_season_games.sort_values(by=['GAME_DATE', 'GAME_ID'], inplace=True)

In [6]:
list(raw_season_games.columns)

['SEASON_ID',
 'TEAM_ID',
 'TEAM_ABBREVIATION',
 'TEAM_NAME',
 'GAME_DATE',
 'MATCHUP',
 'WL',
 'MIN',
 'FGM',
 'FGA',
 'FG_PCT',
 'FG3M',
 'FG3A',
 'FG3_PCT',
 'FTM',
 'FTA',
 'FT_PCT',
 'OREB',
 'DREB',
 'REB',
 'AST',
 'STL',
 'BLK',
 'TOV',
 'PF',
 'PTS',
 'PLUS_MINUS',
 'W_L',
 'W_L_CUM',
 'W_L_L10_CUM',
 'SEASON',
 'LOCATION',
 'UNIQUE_MATCHUP',
 'FGM_ML5',
 'FGA_ML5',
 'FG_PCT_ML5',
 'FG3M_ML5',
 'FG3A_ML5',
 'FG3_PCT_ML5',
 'FTM_ML5',
 'FTA_ML5',
 'FT_PCT_ML5',
 'OREB_ML5',
 'DREB_ML5',
 'REB_ML5',
 'AST_ML5',
 'STL_ML5',
 'BLK_ML5',
 'TOV_ML5',
 'PF_ML5',
 'PTS_ML5',
 'PLUS_MINUS_ML5',
 'W_L_ML5']

In [ ]:

def filter(df):
    return (df.TEAM_ID == 1610612755) & (df.SEASON_ID.str[-4:] == '2017')
    
raw_season_games[filter(raw_season_games)].head(18)

## Season games

In [3]:
season_games = pd.read_feather(config.SEASON_GAMES_DS)
season_games.set_index(["GAME_ID"], inplace=True)
season_games.sort_values(by=['GAME_DATE', 'GAME_ID'], inplace=True)

In [ ]:
def filter(df):
    return ((df.TEAM_ID_HOME == 1610612755) | (df.TEAM_ID_AWAY == 1610612755)) & (df.SEASON_ID.str[-4:] == '2017')
    
#season_games[['PTS_HOME', 'MATCHUP_HOME', 'MATCHUP_AWAY', 'PTS_AWAY', 'HOME_POINT_SPREAD']][filter(season_games)].head(18)
season_games[filter(season_games)].head(18)


In [ ]:
sg.create_calculate_fields(season_games)
#season_games['UNIQUE_MATCHUP_AWAY']

In [ ]:
list(season_games.columns)

# Matchups games

In [ ]:
def matchup_field(row):
    if " vs. " in row.MATCHUP:
        return '-'.join(
            sorted(
                row.MATCHUP.split(" vs. ")
            ))
    else:
        return '-'.join(
            sorted(
                row.MATCHUP.split(" @ ")
            ))                                 

def matchup_field_by_id(row):
    #isplay(row.name)
    game_df = raw_season_games.loc[row.name]
    #ther_team_id = game_df[game_df.TEAM_ID != row.TEAM_ID]['TEAM_ID'].values[0]
    return '-'.join(map(str,sorted(game_df.TEAM_ID.values.tolist())))
#'-'.join(sorted(['BKN', 'ATL']))    
raw_season_games["UNIQUE_MATCHUP"] = raw_season_games.apply(lambda row: matchup_field_by_id(row), axis=1)
#                                                   '-'.join(
#                                                       sorted(
#                                                           [row.TEAM_ABBREVIATION_HOME
#                                                            , row.TEAM_ABBREVIATION_AWAY])),
#                                                       ,axis=1)

"PHI @ ORL".split(" @ ") + "PHI vs. ORL".split(" vs. ")
"PHI @ ORL".split(" @ ") + "PHI @ ORL".split(" vs. ")
" vs. " in "PHI @ ORL"
len(raw_season_games["UNIQUE_MATCHUP"].unique())

# Test transformations

In [100]:
cols = ['SEASON_ID',
 'TEAM_ID',
 'TEAM_ABBREVIATION',
 'TEAM_NAME',
 'GAME_DATE',
 'MATCHUP',
 'WL',
 'FGM','FGM_MEAN',
 'FGA','FGA_MEAN',
 'FG_PCT','FG_PCT_MEAN',
 'FG3M','FG3M_MEAN',
 'FG3A','FG3A_MEAN',
 'FG3_PCT','FG3_PCT_MEAN',
 'FTM','FTM_MEAN',
 'FTA','FTA_MEAN',
 'FT_PCT','FT_PCT_MEAN',
 'OREB','OREB_MEAN',
 'DREB','DREB_MEAN',
 'REB','REB_MEAN',
 'AST','AST_MEAN',
 'STL','STL_MEAN',
 'BLK','BLK_MEAN',
 'TOV','TOV_MEAN',
 'PF','PF_MEAN',
 'PTS','PTS_MEAN',
 'PLUS_MINUS','PLUS_MINUS_MEAN',
 'W_L',
 'W_L_CUM',
 'W_L_L10_CUM',
 'SEASON',
 'LOCATION']

def expanding_mean(x):
    return x.rolling(window=82, min_periods=0).mean()

In [ ]:
df_17 = raw_season_games[raw_season_games.SEASON_ID.str[-4:] == '2017']
df_18  = raw_season_games[raw_season_games.SEASON_ID.str[-4:] == '2018']
pd.concat([df_17, df_18])
#leaguegamelog.LeagueGameLog(season_type_all_star="Regular Season"
#                                           , season=2017).get_data_frames()[0] \
#                                            .set_index('GAME_ID').sort_values(by=['GAME_DATE'])
#.set_index('GAME_ID', inplace=True)
#raw_season_games

In [4]:
# next_season = leaguegamelog.LeagueGameLog(season_type_all_star="Regular Season"
#                                        , season="2017-18").get_data_frames()[0].set_index('GAME_ID').sort_values(by=['GAME_DATE'])

#cond = (raw_season_games.SEASON_ID.str[-4:] == '2018')
cond = ((raw_season_games.TEAM_ABBREVIATION == 'MIL') | (raw_season_games.TEAM_ABBREVIATION == 'ATL')) & (raw_season_games.SEASON_ID.str[-4:] == '2018')
next_season = raw_season_games[cond]
 #
#next_season.reset_index()
#.sort_values(by=['GAME_DATE', 'GAME_ID'])

#grouped = next_season.groupby(by=["TEAM_ID", "GAME_ID"]).count()

#0021800023 4200
print(next_season.loc['0021800003']["FGM"])
res = next_season.groupby(by=["TEAM_ID"])[['FGM']].expanding().mean().groupby(level=0).shift(1)
#.reset_index(level=0)
#res.loc['0021800023']["FGM"]
res

# next_season[['MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB',
#              'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS']] \
# .groupby(next_season["TEAM_ID"], group_keys=False).shift(1) \
#             .rolling(window=82, min_periods=0).mean()
#.apply(expanding_mean)
#season_games_mean
#next_season = pd.merge(next_season, season_games_mean, suffixes=['', '_MEAN'],
#                                on=['GAME_ID'])
# next_season = pd.concat([next_season, season_games_mean], axis=1, sort=False)
# next_season[next_season.TEAM_ABBREVIATION == 'MIL'][cols].head(300)


42


FGM
TEAM_ID    GAME_ID              
1610612737 0021800007        NaN
           0021800020  41.000000
           0021800036  41.000000
           0021800052  43.333333
           0021800076  41.750000
           0021800088  38.800000
           0021800097  38.166667
           0021800114  39.000000
           0021800128  39.375000
           0021800149  40.111111
           0021800155  40.200000
           0021800169  39.909091
           0021800189  39.583333
           0021800201  39.692308
           0021800214  39.714286
           0021800226  39.200000
           0021800243  38.687500
           0021800254  39.058824
           0021800268  39.222222
           0021800288  39.000000
           0021800302  39.550000
           0021800306  39.476190
           0021800325  39.136364
           0021800344  39.304348
           0021800357  39.500000
           0021800380  39.560000
           0021800412  39.576923
           0021800420  39.703704
           0021800436  39.571429
           0021800452  39.827586
           0021800472  39.733333
           0021800486  39.838710
           0021800504  39.718750
           0021800522  39.878788
           0021800529  40.088235
           0021800542  40.171429
           0021800556  40.194444
           0021800572  40.189189
           0021800587  40.105263
           0021800602  40.179487
           0021800609  40.175000
           0021800621  40.073171
           0021800640  40.190476
           0021800653  40.162791
           0021800683  40.522727
           0021800695  40.511111
           0021800712  40.478261
           0021800734  40.574468
           0021800748  40.604167
           0021800762  40.673469
           0021800773  40.620000
           0021800784  40.588235
           0021800791  40.634615
           0021800813  40.735849
           0021800827  40.648148
           0021800837  40.745455
           0021800848  40.678571
           0021800865  40.666667
           0021800877  40.534483
           0021800883  40.576271
           0021800903  40.533333
           0021800912  40.508197
           0021800928  40.596774
           0021800946  40.920635
           0021800953  40.937500
           0021800968  40.892308
           0021800987  40.818182
           0021800995  40.791045
           0021801016  40.897059
           0021801034  41.057971
           0021801046  41.085714
           0021801060  41.042254
           0021801076  41.013889
           0021801088  41.013699
           0021801113  41.121622
           0021801131  41.160000
           0021801145  41.118421
           0021801162  41.259740
           0021801168  41.282051
           0021801181  41.367089
           0021801202  41.362500
           0021801220  41.345679
1610612749 0021800003        NaN
           0021800023  42.000000
           0021800042  43.000000
           0021800059  43.666667
           0021800071  43.000000
           0021800081  44.200000
           0021800092  43.666667
           0021800115  43.857143
           0021800133  43.500000
           0021800152  43.777778
           0021800166  43.400000
           0021800175  44.181818
           0021800187  44.416667
           0021800207  44.692308
           0021800223  44.428571
           0021800245  44.600000
           0021800259  44.312500
           0021800275  45.058824
           0021800284  44.888889
           0021800294  45.105263
           0021800310  44.700000
           0021800330  44.619048
           0021800361  44.681818
           0021800377  44.391304
           0021800388  44.041667
           0021800395  43.720000
           0021800407  43.653846
           0021800423  43.444444
           0021800443  43.321429
           0021800462  43.344828
           0021800473  43.333333
           0021800482  43.258065
           0021800497  42.875000
           0021800513  42.787879
           0021800527  42.764706
           0021800550  42.885714
           0021800572  43.000000
           0021800580  43.324324
   

In [ ]:
# .reset_index(level=0)
# season_games_mean
# season_games_mean.isnull().sum().sum()
#next_season = pd.merge(next_season, season_games_mean, suffixes=['', '_MEAN'], on=['GAME_ID', 'TEAM_ID'])

# season_l10_games_mean = next_season.groupby(by=["TEAM_ID"]).shift(1, fill_value=0)[
#             ['MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB',
#              'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS']] \
#             .rolling(window=10, min_periods=1).mean()
# season_l10_games_mean = season_l10_games_mean.assign(key=next_season['TEAM_ID'])
# season_l10_games_mean

#result = test_season_games.groupby(by=["TEAM_ID", "UNIQUE_MATCHUP"])[['W_L']]\
#.expanding().sum().reset_index(level=0)

#print(next_season[(next_season.index == '0021700017') & (next_season.TEAM_ID == 1610612737)].iloc[0])
#cond = (result.TEAM_ID == 1610612737) & (result.UNIQUE_MATCHUP == '1610612737-1610612738')
#result[cond]

#.reset_index(level=0)
#.agg({'A' : ['sum', 'min'], 'B' : ['min', 'max']})
#next_season
# result
# compare_df = pd.DataFrame()
# print(result["TEAM_ID"].shape)
# print(next_season["TEAM_ID"].shape)
# compare_df['team_id'] = next_season["TEAM_ID"]
# compare_df['result_team_id'] = result["TEAM_ID"]

# comparison = result["TEAM_ID"].values == next_season["TEAM_ID"].values
# equal_arrays = comparison.all()

# np.setdiff1d(result["TEAM_ID"].values, next_season["TEAM_ID"].values)
# diff = result["TEAM_ID"].values - next_season["TEAM_ID"].values
# diff
# #compare_df['different'] = np.where( == , True, False)
# #compare_df
raw_season_games[raw_season_games.UNIQUE_MATCHUP == '1610612737-1610612738']

# Fetch and transform season games

In [ ]:
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.endpoints import leaguegamelog
import model.dataset.data as data

seasons = data.load_seasons().SEASON.unique()
# season_games = leaguegamelog.LeagueGameLog(season_type_all_star="Regular Season"
#                                             , season=seasons[-11]).get_data_frames()[0].sort_values(by=['SEASON_ID', 'GAME_DATE', 'GAME_ID'])
season_games = pd.DataFrame()
raw_season_games = pd.DataFrame()
for season in seasons[-11:]:
    next_season = leaguegamelog.LeagueGameLog(season_type_all_star="Regular Season"
                                           , season=season).get_data_frames()[0] \
                                            .set_index('GAME_ID').sort_values(by=['GAME_DATE'])
                                            #.sort_values(by=['SEASON_ID', 'GAME_DATE', 'GAME_ID'])
    raw_season_games = pd.concat([raw_season_games, next_season])
    
    next_season.dropna(inplace=True)
    next_season["W_L"] = np.where(next_season['WL'] == 'W', 1, -1)
    season_games_sum = next_season.groupby(by=["TEAM_ID"])[['W_L']]\
    .expanding().sum().reset_index(level=0)
    next_season = pd.merge(next_season, season_games_sum, suffixes=['', '_CUM'], on=['GAME_ID', 'TEAM_ID'])
    
    season_games_mean = next_season.groupby(by=["TEAM_ID"])[['MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS']]\
    .expanding().mean().reset_index(level=0)
    next_season = pd.merge(next_season, season_games_mean, suffixes=['', '_MEAN'], on=['GAME_ID', 'TEAM_ID'])
    
    next_season.groupby(by=["TEAM_ID"])[['W_L', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS']]\
    .rolling(window=10, min_periods=1).median().reset_index(level=0)
    next_season = pd.merge(next_season, season_games_mean, suffixes=['', '_L10'], on=['GAME_ID', 'TEAM_ID'])
    
    season_home_rows = next_season[next_season.MATCHUP.str.contains('vs.')]
    season_away_rows = next_season[next_season.MATCHUP.str.contains('@')]
    # Join every row to all others with the same game ID.
    joined = pd.merge(season_home_rows, season_away_rows, suffixes=['_HOME', '_AWAY'],
                      on=['SEASON_ID', 'GAME_ID', 'GAME_DATE'])
    # Filter out any row that is joined to itself.
    result = joined[joined.TEAM_ID_HOME != joined.TEAM_ID_AWAY]
    season_games = pd.concat([season_games, result])

In [ ]:
pd.options.display.max_rows = 999


season_games[season_games.isnull().any(axis=1)]

In [ ]:
#season_games['SEASON_YEAR'] = season_games.SEASON_ID.str[1:]
season_games.drop(columns=['VIDEO_AVAILABLE_HOME', 'VIDEO_AVAILABLE_AWAY'], axis=1, inplace=True)

# Test fetched season games

In [ ]:
def create_filter(team, season_id, game_date):
    return (season_games.GAME_DATE < game_date) & \
            (season_games.SEASON_ID == season_id) & \
            ((season_games.TEAM_ID_HOME == team) | (season_games.TEAM_ID_AWAY == team))
    

In [ ]:
row.index

In [ ]:


row = season_games.iloc[200]
print(row)
home = row.TEAM_ID_HOME
away = row.TEAM_ID_AWAY
season_id = row.SEASON_ID
game_date = row.GAME_DATE

home_games = season_games[create_filter(home, season_id, game_date)]
away_games = season_games[create_filter(home, season_id, game_date)]

home_games.columns

#season_games.expanding().mean()


# Testing teamgamelogs endpoint

In [ ]:
from nba_api.stats.endpoints import teamgamelogs

# Query for games where the Celtics were playing
gamefinder = teamgamelogs.TeamGameLogs(team_id_nullable=celtics_id)
all_games = teamgamelogs.TeamGameLogs(season_type_nullable="Regular Season"
                                          , season_nullable='2010-11').get_data_frames()[0]
teamgamelogs.SeasonNullable.current_season
# The first DataFrame of those returned is what we want.
all_games.head()


In [ ]:
all_games.groupby(all_games.SEASON_YEAR)[['GAME_ID']].count()
#.loc['2015':]